# Entropy pruning

## IMPORTS

In [7]:
import os
import tempfile

import numpy as np
import pandas as pd

from configparser import ConfigParser
os.environ["TF_USE_LEGACY_KERAS"]="1"

from tf_keras import losses,optimizers, models

import tensorflow_model_optimization as tfmot

from utils import load_dataset, reformat_image, entropy_prune_model
from utils.Objects import EntropyPruningSurgeon, PrunedModel
from utils.Plots import plot_entropy_accuracy_threshold

---

## CONSTANTS

In [8]:
config = ConfigParser()
config.read("config.ini")

BATCH_SIZE = int(config["BATCHES"]["BATCH_SIZE"])

IMG_SIZE = (int(config["IMAGES"]["IMG_SIZE"]), int(config["IMAGES"]["IMG_SIZE"]))

MODEL_DIRECTORY = config["MODEL"]["MODEL_DIRECTORY"]
METRICS_DIRECTORY = config["MODEL"]["METRICS_DIRECTORY"]
WEIGHTS_DIRECTORY = config["MODEL"]["WEIGHTS_DIRECTORY"]
ENTROPY_METRICS = os.path.join(METRICS_DIRECTORY, config["MODEL"]["ENTROPY_METRICS"])
HORSE_AND_HUMAN_MODEL = os.path.join(MODEL_DIRECTORY, config["MODEL"]["HORSE_AND_HUMAN_MODEL"])
ENTROPY_HORSE_AND_HUMAN_WEIGHTS = os.path.join(WEIGHTS_DIRECTORY, config["MODEL"]["ENTROPY_HORSE_AND_HUMAN_WEIGHTS"])

## Import the dataset

In [9]:
train_examples, validation_examples, num_examples, num_classes, class_names = load_dataset('horses_or_humans', 70)

INFO:absl:Load dataset info from C:\Users\neyen\tensorflow_datasets\horses_or_humans\3.0.0
INFO:absl:Creating a tf.data.Dataset reading 2 files located in folders: C:\Users\neyen\tensorflow_datasets\horses_or_humans\3.0.0.
INFO:absl:Creating a tf.data.Dataset reading 1 files located in folders: C:\Users\neyen\tensorflow_datasets\horses_or_humans\3.0.0.
INFO:absl:Constructing tf.data.Dataset horses_or_humans for split ('train[:70%]', 'train[70%:]'), from C:\Users\neyen\tensorflow_datasets\horses_or_humans\3.0.0


In [10]:
train_batches = train_examples.cache().shuffle(num_examples // 4).map(reformat_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(reformat_image).batch(BATCH_SIZE).prefetch(1)

In [11]:
# Choose the best entropy level
k_entropies = np.arange(0.019, 0.031, 0.001)
metric_list = []

for k in k_entropies:
    # Load in the best saved model
    model_1 = models.load_model("model_experiments/custom_vgg16.keras")
    pruned_model: PrunedModel = entropy_prune_model(model_1, 20, k, train_batches, validation_batches, BATCH_SIZE)
    metrics = pruned_model.metrics
    val_loss = metrics["val_loss"]
    val_accuracy = metrics["val_accuracy"]

    print(f"Entropy={k}, \t {val_loss=}, \t {val_accuracy=}")
    metric_list.append(metrics)

# Create a dataframe of the values obtained
df = pd.DataFrame(metric_list)
df.to_csv(ENTROPY_METRICS)
df

Deleting 0/64 channels from layer: block1_conv1
Deleting 0/64 channels from layer: block1_conv2
Deleting 2/128 channels from layer: block2_conv1
Deleting 0/128 channels from layer: block2_conv2
Deleting 0/256 channels from layer: block3_conv1
Deleting 0/256 channels from layer: block3_conv2
Deleting 0/256 channels from layer: block3_conv3
Deleting 0/512 channels from layer: block4_conv1
Deleting 0/512 channels from layer: block4_conv2
Deleting 0/512 channels from layer: block4_conv3
Deleting 0/512 channels from layer: block5_conv1
Deleting 0/512 channels from layer: block5_conv2
Deleting 0/512 channels from layer: block5_conv3
Epoch 1/20



KeyboardInterrupt: 

In [22]:
data = pd.read_csv(ENTROPY_METRICS)
plot_entropy_accuracy_threshold(data)

FileNotFoundError: [Errno 2] No such file or directory: 'metrics_experiments\\entropy_metrics.csv'

## Final Model Pruning Entropy

In [16]:
final_threshold = 0.026
final_model = models.load_model("model_experiments/custom_vgg16.keras")
pruning = EntropyPruningSurgeon(model=final_model, threshold=final_threshold)
final_pruned_model = pruning.run()
final_pruned_model.summary()

Deleting 0/64 channels from layer: block1_conv1
Deleting 0/64 channels from layer: block1_conv2
Deleting 3/128 channels from layer: block2_conv1
Deleting 0/128 channels from layer: block2_conv2
Deleting 0/256 channels from layer: block3_conv1
Deleting 0/256 channels from layer: block3_conv2
Deleting 0/256 channels from layer: block3_conv3
Deleting 237/512 channels from layer: block4_conv1
Deleting 98/512 channels from layer: block4_conv2
Deleting 99/512 channels from layer: block4_conv3
Deleting 39/512 channels from layer: block5_conv1
Deleting 48/512 channels from layer: block5_conv2
Deleting 109/512 channels from layer: block5_conv3
Model: "model_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 input_40 (InputLayer)       multiple                  0         
         

In [17]:
# Test the model
final_pruned_model.compile(loss=losses.SparseCategoricalCrossentropy(),
              optimizer = optimizers.Adam(learning_rate=0.0001),
              metrics=['accuracy'])
final_pruned_model.evaluate(validation_batches)
final_pruned_model.save_weights(ENTROPY_HORSE_AND_HUMAN_WEIGHTS)

 3/10 [========>.....................] - ETA: 28s - loss: 4.7112 - accuracy: 0.7604

KeyboardInterrupt: 

In [196]:
model_for_export = tfmot.sparsity.keras.strip_pruning(final_pruned_model)

_, pruned_keras_file = tempfile.mkstemp('.h5')
models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)
print('Saved pruned Keras model to:', pruned_keras_file)

C:\Users\neyen\AppData\Local\Temp\ipykernel_71452\369930696.py:4: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native TF-Keras format, e.g. `model.save('my_model.keras')`.
  models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)


Saved pruned Keras model to: C:\Users\neyen\AppData\Local\Temp\tmpjh1lij65.h5
